In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"
model_dir = "model/"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=model_dir)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    cache_dir=model_dir
)

print(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [2]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [6]:
prompt = "将一个关于大学生的笑话"
messages = [
    {"role":"system","content":"You are JaLmg, a helpful assistant."},
    {"role":"user","content":prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(text)

<|im_start|>system
You are JaLmg, a helpful assistant.<|im_end|>
<|im_start|>user
将一个关于大学生的笑话<|im_end|>
<|im_start|>assistant



In [7]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print(model_inputs)

{'input_ids': tensor([[151644,   8948,    198,   2610,    525,  22871,     43,  12311,     11,
            264,  10950,  17847,     13, 151645,    198, 151644,    872,    198,
          44063,  46944, 101888, 104422,   9370, 109959, 151645,    198, 151644,
          77091,    198]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]])}


In [8]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs["input_ids"], generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

<think>
好的，用户让我提供一个关于大学生的笑话。首先，我需要确保笑话适合所有年龄段，避免任何可能引起不适的内容。大学生这个群体可能有各种有趣的背景，但要注意幽默的表达方式。

接下来，我得考虑笑话的结构。通常，笑话会有一个问题或情景，然后通过反转或意想不到的方式解决。比如，大学生可能遇到一些常见的问题，但用幽默的方式表达。

然后，我需要检查笑话的连贯性和逻辑性。确保笑话不会让人感到困惑或不适。同时，要保持语言简洁，让读者容易理解。

另外，还要注意笑话的长度，不要太长，适合快速阅读。可能需要调整句子的结构，使其更流畅。

最后，确保笑话符合大学生的常见场景，比如校园生活、学习压力等，这样更容易引起共鸣。同时，加入一些生动的细节，比如表情、动作，可以让笑话更生动有趣。

总结一下，我需要构思一个简单明了、有反转和幽默感的笑话，围绕大学生的生活或问题展开，确保内容健康积极，适合目标读者。
</think>

当然！这是一个关于大学生的幽默笑话：

**"为什么大学生要考研究生？因为……因为……因为……"**

（突然，一个男生从教室里跑出来，手里拿着一个发光的盒子，嘴里喊着："为什么大学生要考研究生？因为……因为……因为……"）

（镜头拉远，一个穿着校服的男生站在教室门口，手里拿着一个发光的盒子，笑得大笑）  

**"因为……因为……因为……"**  

（镜头定格，男生露出开心的笑容，盒子里掉出一张写着"**考研究生**"的纸条。）  

**笑点解析**：  
- 通过反转（盒子里的纸条）制造悬念，  
- 用"为什么"引出荒诞的逻辑，  
- 结尾的"考研究生"点明主题，  
- 用表情和动作增强画面感。


In [9]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=512,
    do_sample=False
)

messages = [
    {"role":"user","content":"请介绍一下你自己"}
]

output = generator(messages)
print(output[0]["generated_text"])

Device set to use cpu
d:\Anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
d:\Anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
d:\Anaconda\Lib\site-packages\transformers\generation\configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


<think>
好的，用户让我介绍一下自己。首先，我需要确定用户的需求是什么。他们可能是在测试我的能力，或者想了解我的背景，或者只是好奇我的身份。作为AI助手，我应该保持自然和友好的语气，同时提供有用的信息。

接下来，我需要考虑用户可能的背景。他们可能是学生、工作人士，或者对AI感兴趣的人。不同的用户可能需要不同的回答。比如，学生可能需要更专业的介绍，而职场人士可能更关注功能和优势。

然后，我需要确保回答的信息准确且全面。要涵盖我的主要功能，比如知识库、对话历史、多语言支持等。同时，要强调我的优点，比如快速响应、灵活调整、隐私保护等，以展示我的可靠性。

还要注意用户可能的深层需求。他们可能希望了解如何使用我的服务，或者如何与我互动。因此，在回答中可以提供一些使用建议，比如如何开始对话，或者如何优化我的回答。

最后，保持回答简洁明了，避免使用过于技术化的术语，让所有用户都能轻松理解。同时，保持礼貌和专业的形象，确保信息传达清晰。
</think>

你好！我是你的AI助手，可以帮你解答各种问题，从学习到工作，从日常事务到创意灵感。如果你有任何问题或需要帮助，请随时告诉我！😊


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B", cache_dir="./models")

label = [151667, 106287, 3837, 20002, 56007, 100146, 103133, 33108, 100414, 113700, 100200, 24339, 109226, 3837, 100626, 103133, 113700, 100200, 24339, 18493, 101921, 100200, 99328, 100376, 100145, 101047, 100780, 1773, 101140, 3837, 35946, 85106, 104843, 100158, 100376, 99667, 105166, 70538, 1773, 100414, 113700, 100376, 102119, 111198, 29991, 100692, 9370, 100376, 26898, 99812, 3837, 68536, 106146, 99569, 32948, 105149, 100376, 1773, 101886, 3837, 104136, 13343, 85106, 100374, 99912, 116541, 3837, 66394, 100678, 103133, 100240, 115946, 3407, 104019, 101118, 100200, 24339, 18493, 101921, 100376, 100145, 101047, 100780, 3837, 101912, 101999, 104951, 5373, 101048, 102439, 5373, 101902, 100138, 100355, 49567, 1773, 87267, 85106, 114336, 66394, 3837, 101912, 101924, 102439, 100153, 5373, 100182, 100817, 54542, 5373, 113020, 57621, 104397, 49567, 3837, 100001, 108080, 103133, 113700, 100200, 24339, 36407, 99553, 100011, 9370, 100376, 102724, 3407, 100161, 3837, 30534, 103944, 102104, 100166, 104542, 3837, 60726, 108568, 107051, 3837, 87256, 100700, 66394, 103133, 100240, 104149, 3837, 91572, 11622, 20002, 100047, 101128, 109824, 3837, 101153, 102767, 104380, 32108, 3837, 77288, 107919, 102188, 1773, 87267, 85106, 101071, 107189, 114078, 105386, 27442, 3837, 101912, 100200, 24339, 100622, 100376, 99667, 106681, 3837, 101034, 99652, 106961, 100376, 99667, 9370, 72064, 3837, 101912, 57218, 100059, 24339, 5373, 69721, 24339, 9370, 106809, 99659, 8997, 151668, 715, 92293, 99178, 113700, 100200, 24339, 57218, 100414, 113700, 100200, 24339, 104396, 102665, 101321, 3837, 100414, 113700, 100200, 24339, 65278, 63367, 99599, 106624, 100776, 104016, 9370, 23031, 14009, 100200, 24339, 527, 57191, 14009, 100200, 24339, 99548, 527, 109734, 9370, 100376, 3837, 68536, 103133, 113700, 100200, 24339, 104442, 55338, 101921, 100200, 99328, 100376, 100145, 9370, 100376, 101931, 9370, 59743, 24641, 3837, 99652, 100409, 99599, 100376, 100365, 104111, 100376, 99667, 1773, 103133, 113700, 100200, 24339, 99902, 100630, 23031, 14009, 100200, 24339, 527, 109734, 9370, 100376, 3837, 114735, 92894, 23990, 22243, 100376, 5373, 100059, 102648, 5373, 104341, 104136, 5373, 99667, 103638, 49567, 1773, 18493, 101921, 100200, 99328, 100376, 100145, 15946, 3837, 103133, 113700, 100200, 24339, 102889, 34187, 100185, 100780, 3837, 99652, 103944, 34187, 100200, 104799, 100376, 104951, 3837, 109357, 100200, 100138, 107474, 100355, 3837, 101048, 34187, 101380, 99722, 102439, 3837, 104979, 17714, 100200, 112045, 105647, 104257, 104653, 109221, 33108, 100419, 99896, 1773, 151643]

print(tokenizer.decode(label))

<think>嗯，用户问的是实质和形式意义上的卫生法的区别，还有实质意义上的卫生法在调整卫生社会法律关系中的角色。首先，我需要回忆一下法律部门的基本分类。形式意义上的法律通常指的是名称明确的法律文件规定，而不仅仅是某部特定法律。因此，解释时需要结合实际应用场景，说明为什么实质意义更重要。

还要考虑卫生法在调整法律关系中的角色，比如维护秩序、保障权益、促进政策实施等。可能需要举例说明，比如患者权益保护、医疗事故处理、公共卫生事件应对等，这些都需要实质意义上的卫生法来提供全面的法律框架。

最后，要确保回答结构清晰，先区分两者，再详细说明实质意义的作用，同时用用户容易理解的语言，避免过于学术化，但又要准确。可能需要检查是否有遗漏的关键点，比如卫生法作为法律部门的地位，以及它与其他法律部门的联系，比如与行政法、民法的交叉部分。
</think> 
 实质意义上的卫生法与形式意义上的卫生法的主要区别在于，形式意义上的卫生法特指国家立法机关制定的以‘卫生法’或‘卫生法典’命名的法律，而实质意义上的卫生法是指所有调整卫生社会法律关系的法律规范的总称，它属于国家法律体系的一个法律部门。实质意义上的卫生法不仅包括以‘卫生法’命名的法律，还包括其他单行法律、行政法规、司法解释、部门规章等。在调整卫生社会法律关系中，实质意义上的卫生法扮演了核心角色，它确保了卫生领域的法律秩序，促进了卫生政策的有效实施，保障了公众健康权益，同时也为卫生法学的研究提供了丰富的素材和实践基础。<|endoftext|>


In [3]:
print(tokenizer.decode([151667]))

<think>


In [4]:
input = [27, 91, 8948, 91, 397, 56568, 101909, 104316, 101057, 3837, 112735, 100345, 20002, 103936, 3837, 107485, 106646, 104107, 111423, 89393, 91, 408, 91, 397, 27, 91, 872, 91, 397, 103998, 3837, 104100, 110050, 103133, 113700, 100200, 24339, 57218, 100414, 113700, 100200, 24339, 104139, 102665, 3837, 101034, 103133, 113700, 100200, 24339, 18493, 101921, 100200, 99328, 100376, 100145, 15946, 102889, 34187, 99245, 100780, 11319, 27, 91, 408, 91, 397, 27, 91, 77091, 91, 397, 151667, 106287, 3837, 20002, 56007, 100146, 103133, 33108, 100414, 113700, 100200, 24339, 109226, 3837, 100626, 103133, 113700, 100200, 24339, 18493, 101921, 100200, 99328, 100376, 100145, 101047, 100780, 1773, 101140, 3837, 35946, 85106, 104843, 100158, 100376, 99667, 105166, 70538, 1773, 100414, 113700, 100376, 102119, 111198, 29991, 100692, 9370, 100376, 26898, 3837, 101912, 26940, 100200, 24339, 25067, 57191, 26940, 100200, 24339, 99548, 87243, 68536, 103133, 100240, 9370, 87267, 33126, 100789, 3837, 102994, 55338, 78556, 100376, 101931, 3407, 101889, 3837, 20002, 104496, 9370, 100200, 99328, 100376, 100145, 3837, 99730, 104442, 100182, 5373, 113020, 106483, 101047, 100646, 100376, 100145, 3837, 101912, 99286, 99811, 100145, 5373, 100200, 115008, 57218, 108097, 104612, 104008, 1773, 103133, 113700, 100200, 24339, 104067, 100622, 100185, 3837, 87267, 104404, 99652, 102102, 34187, 100646, 106221, 3837, 103944, 100001, 100145, 18830, 24339, 30440, 99539, 3407, 104326, 3837, 35946, 85106, 81167, 77540, 106017, 102665, 64471, 99373, 101321, 29991, 33108, 100414, 3837, 99998, 18830, 33126, 113879, 9370, 100166, 102478, 1773, 101912, 3837, 100414, 113700, 87267, 100009, 23990, 105042, 100376, 3837, 68536, 103133, 9370, 104428, 101908, 100376, 100365, 106979, 3837, 102298, 99604, 115972, 9370, 101931, 3837, 29524, 100376, 5373, 102648, 5373, 103638, 49567, 3407, 101948, 3837, 20002, 20412, 103998, 3837, 87267, 33126, 100020, 99912, 99892, 101047, 100376, 86119, 3837, 101912, 18493, 104595, 107933, 104011, 9370, 100376, 86119, 3837, 87267, 85106, 103346, 103133, 113700, 100200, 24339, 101047, 100646, 99812, 3837, 68536, 106146, 99569, 32948, 105149, 100376, 1773, 101886, 3837, 104136, 13343, 85106, 100374, 99912, 116541, 3837, 66394, 100678, 103133, 100240, 115946, 3407, 104019, 101118, 100200, 24339, 18493, 101921, 100376, 100145, 101047, 100780, 3837, 101912, 101999, 104951, 5373, 101048, 102439, 5373, 101902, 100138, 100355, 49567, 1773, 87267, 85106, 114336, 66394, 3837, 101912, 101924, 102439, 100153, 5373, 100182, 100817, 54542, 5373, 113020, 57621, 104397, 49567, 3837, 100001, 108080, 103133, 113700, 100200, 24339, 36407, 99553, 100011, 9370, 100376, 102724, 3407, 100161, 3837, 30534, 103944, 102104, 100166, 104542, 3837, 60726, 108568, 107051, 3837, 87256, 100700, 66394, 103133, 100240, 104149, 3837, 91572, 11622, 20002, 100047, 101128, 109824, 3837, 101153, 102767, 104380, 32108, 3837, 77288, 107919, 102188, 1773, 87267, 85106, 101071, 107189, 114078, 105386, 27442, 3837, 101912, 100200, 24339, 100622, 100376, 99667, 106681, 3837, 101034, 99652, 106961, 100376, 99667, 9370, 72064, 3837, 101912, 57218, 100059, 24339, 5373, 69721, 24339, 9370, 106809, 99659, 8997, 151668, 715, 92293, 99178, 113700, 100200, 24339, 57218, 100414, 113700, 100200, 24339, 104396, 102665, 101321, 3837, 100414, 113700, 100200, 24339, 65278, 63367, 99599, 106624, 100776, 104016, 9370, 23031, 14009, 100200, 24339, 527, 57191, 14009, 100200, 24339, 99548, 527, 109734, 9370, 100376, 3837, 68536, 103133, 113700, 100200, 24339, 104442, 55338, 101921, 100200, 99328, 100376, 100145, 9370, 100376, 101931, 9370, 59743, 24641, 3837, 99652, 100409, 99599, 100376, 100365, 104111, 100376, 99667, 1773, 103133, 113700, 100200, 24339, 99902, 100630, 23031, 14009, 100200, 24339, 527, 109734, 9370, 100376, 3837, 114735, 92894, 23990, 22243, 100376, 5373, 100059, 102648, 5373, 104341, 104136, 5373, 99667, 103638, 49567, 1773, 18493, 101921, 100200, 99328, 100376, 100145, 15946, 3837, 103133, 113700, 100200, 24339, 102889, 34187, 100185, 100780, 3837, 99652, 103944, 34187, 100200, 104799, 100376, 104951, 3837, 109357, 100200, 100138, 107474, 100355, 3837, 101048, 34187, 101380, 99722, 102439, 3837, 104979, 17714, 100200, 112045, 105647, 104257, 104653, 109221, 33108, 100419, 99896, 1773, 151643]
print(tokenizer.decode(input))

<|system|>
你是一个医学专家，你需要根据用户的问题，给出带有思考的回答。<|end|>
<|user|>
医生，我想了解一下实质意义上的卫生法与形式意义上的卫生法有什么区别，以及实质意义上的卫生法在调整卫生社会法律关系中扮演了什么角色？<|end|>
<|assistant|>
<think>嗯，用户问的是实质和形式意义上的卫生法的区别，还有实质意义上的卫生法在调整卫生社会法律关系中的角色。首先，我需要回忆一下法律部门的基本分类。形式意义上的法律通常指的是名称明确的法律文件，比如《卫生法》或《卫生法典》，而实质意义的可能更广泛，涵盖所有相关法律规范。

然后，用户提到的卫生社会法律关系，应该是指医疗、公共卫生等领域中的各种法律关系，比如医患关系、卫生行政部门与医疗机构的关系等等。实质意义上的卫生法在这里作为核心，可能是因为它整合了各种法律法规，确保这些关系有法可依。

接下来，我需要确认两者的区别是否仅在于名称和形式，还是有更深层次的结构差异。比如，形式意义上的可能只是单一部法律，而实质的则是整个法律体系的一部分，包含不同层级的规范，如法律、法规、规章等。

另外，用户是医生，可能更关注实际应用中的法律问题，比如在临床实践中遇到的法律问题，可能需要引用实质意义上的卫生法中的各种规定，而不仅仅是某部特定法律。因此，解释时需要结合实际应用场景，说明为什么实质意义更重要。

还要考虑卫生法在调整法律关系中的角色，比如维护秩序、保障权益、促进政策实施等。可能需要举例说明，比如患者权益保护、医疗事故处理、公共卫生事件应对等，这些都需要实质意义上的卫生法来提供全面的法律框架。

最后，要确保回答结构清晰，先区分两者，再详细说明实质意义的作用，同时用用户容易理解的语言，避免过于学术化，但又要准确。可能需要检查是否有遗漏的关键点，比如卫生法作为法律部门的地位，以及它与其他法律部门的联系，比如与行政法、民法的交叉部分。
</think> 
 实质意义上的卫生法与形式意义上的卫生法的主要区别在于，形式意义上的卫生法特指国家立法机关制定的以‘卫生法’或‘卫生法典’命名的法律，而实质意义上的卫生法是指所有调整卫生社会法律关系的法律规范的总称，它属于国家法律体系的一个法律部门。实质意义上的卫生法不仅包括以‘卫生法’命名的法律，还包括其他单行法律、行政法规、司法解释、部门规章等。在调